# How concepts in SYNERGY are computed

Concepts, also known as topics, are attached to each OpenAlex record. They are generated with an machine leanring model. You can indetify at least 3 different types of concepts per dataset in SYNERGY:

- The main (level 0) concept of the publication record
- The average main (level 0) concept of all records in the dataset (the search)
- The average main (level 0) concept of all included records in the dataset

See the table at the bottom of this notebook for the topic per dataset

In [1]:
import synergy_dataset as sd
import pandas as pd

In [2]:
# function to standardize concept data

def flatten_concepts(ds, level=0):
    result = []
    for openalex_id, record in ds.to_dict(["concepts"]).items():

        d = {"openalex_id": openalex_id,
             "label_included": record["label_included"]}

        for c in record["concepts"]:
            if c["level"] == level:
                d[c["display_name"]] = c["score"]

        result.append(d)
    
    return pd.DataFrame(result)

In [3]:
keys = [d.name for d in sd.iter_datasets()]
df = pd.concat([flatten_concepts(d) for d in sd.iter_datasets()], keys=keys)
df

openalex_id   
Appenzeller-Herzog_2019 0     https://openalex.org/W1971651073  \
                        1     https://openalex.org/W2060266518   
                        2     https://openalex.org/W1985329347   
                        3     https://openalex.org/W2033425122   
                        4     https://openalex.org/W2174281073   
...                                                        ...   
Wolters_2018            4275  https://openalex.org/W1995096921   
                        4276  https://openalex.org/W2114922669   
                        4277  https://openalex.org/W1657669688   
                        4278  https://openalex.org/W1994490975   
                        4279  https://openalex.org/W1504586396   

                              label_included    Medicine   Chemistry Physics   
Appenzeller-Herzog_2019 0                  0  0.56412965  0.09127462     0.0  \
                        1                  0   0.4624714  0.31808412     NaN   
                        2                  0         0.0   0.8244176     NaN   
                        3                  0   0.5881644         NaN     NaN   
                        4                  0  0.41348755  0.35551745     NaN   
...                                      ...         ...         ...     ...   
Wolters_2018            4275               0  0.91154534         NaN     NaN   
                        4276               0  0.95886254         NaN     NaN   
                        4277               0  0.96129704         NaN     NaN   
                        4278               0   0.9699614         NaN     NaN   
                        4279               0   0.8246919         NaN     NaN   

                                 Biology Psychology Economics   
Appenzeller-Herzog_2019 0            NaN        NaN       NaN  \
                        1     0.36406675        NaN       NaN   
                        2     0.10137832        NaN       NaN   
                        3            NaN        NaN       NaN   
                        4            NaN        NaN       NaN   
...                                  ...        ...       ...   
Wolters_2018            4275         NaN        NaN       NaN   
                        4276         NaN        NaN       NaN   
                        4277         NaN        NaN       NaN   
                        4278         NaN        NaN       NaN   
                        4279         NaN        NaN       NaN   

                             Materials science Philosophy  ...   
Appenzeller-Herzog_2019 0                  NaN        NaN  ...  \
                        1                  NaN        NaN  ...   
                        2                  NaN        NaN  ...   
                        3                  NaN        NaN  ...   
                        4                  NaN        NaN  ...   
...                                        ...        ...  ...   
Wolters_2018            4275               NaN        NaN  ...   
                        4276               NaN        NaN  ...   
                        4277               NaN        NaN  ...   
                        4278               NaN        NaN  ...   
                        4279               NaN        NaN  ...   

                             Political science History Computer science   
Appenzeller-Herzog_2019 0                  NaN     NaN              NaN  \
                        1                  NaN     NaN              NaN   
                        2                  NaN     NaN              NaN   
                        3                  NaN     NaN              NaN   
                        4                  NaN     NaN              NaN   
...                                        ...     ...              ...   
Wolters_2018            4275               NaN     NaN              NaN   
                        4276               NaN     NaN              NaN   
                        4277               NaN  

The following table shows the average concept scores for all records in the dataset. 

In [4]:
concepts_per_dataset = df.drop(["label_included", "openalex_id"], axis=1).astype(float).fillna(0).groupby(level=0).mean()
concepts_per_dataset

,Medicine,Chemistry,Physics,Biology,Psychology,Economics,Materials science,Philosophy,Engineering,Political science,History,Computer science,Sociology,Mathematics,Geology,Business,Art,Geography,Environmental science
Appenzeller-Herzog_2019,0.588982,0.103742,0.002754,0.054124,0.030139,0.000246,0.004020,0.006154,0.000616,0.002405,0.002357,0.006770,0.000000,0.000370,0.000309,0.001908,0.000138,0.001154,0.001068
Bos_2018,0.565684,0.008519,0.004312,0.028526,0.202546,0.000056,0.000249,0.000376,0.000072,0.000096,0.000118,0.015857,0.000069,0.004413,0.000068,0.000109,0.000021,0.000077,0.000067
Brouwer_2019,0.332740,0.006930,0.000627,0.011639,0.451345,0.001146,0.000212,0.004143,0.000596,0.002774,0.001253,0.006507,0.004893,0.001216,0.000184,0.001035,0.001908,0.001189,0.000237
Chou_2003,0.865703,0.005159,0.000069,0.002794,0.011938,0.000102,0.000122,0.000174,0.000111,0.001076,0.000052,0.002323,0.000000,0.000330,0.000000,0.000125,0.000033,0.000090,0.000000
Chou_2004,0.760822,0.009780,0.000445,0.010023,0.056741,0.000225,0.000560,0.000095,0.000701,0.001352,0.000218,0.004468,0.000153,0.000146,0.000068,0.001643,0.000033,0.000079,0.000000
Donners_2021,0.663195,0.053750,0.000000,0.027470,0.002286,0.000324,0.001471,0.002270,0.002108,0.009966,0.000000,0.016382,0.000000,0.001673,0.000000,0.005976,0.002458,0.000747,0.000000
Hall_2012,0.003415,0.001369,0.032076,0.001220,0.001738,0.000374,0.020137,0.000058,0.139658,0.000253,0.000091,0.599865,0.000096,0.055082,0.012524,0.002743,0.000054,0.001584,0.004120
Jeyaraman_2020,0.723767,0.060434,0.000291,0.041080,0.000000,0.000000,0.055064,0.000452,0.001324,0.000000,0.000000,0.003419,0.000000,0.000392,0.000000,0.000179,0.000108,0.000000,0.000000
Leenaars_2019,0.233326,0.420204,0.000652,0.161313,0.064596,0.000053,0.001316,0.000404,0.000146,0.001013,0.000138,0.002701,0.000075,0.000171,0.000025,0.000084,0.000077,0.000055,0.000050
Leenaars_2020,0.839694,0.017233,0.000281,0.023396,0.000667,0.000301,0.001126,0.000470,0.000206,0.000479,0.000014,0.003330,0.000029,0.001062,0.000000,0.000488,0.000247,0.000094,0.000000


Same for the inclusions per dataset

In [5]:
concepts_per_dataset_incl = df[df["label_included"] == 1].drop(["label_included", "openalex_id"], axis=1).astype(float).fillna(0).groupby(level=0).mean()
concepts_per_dataset_incl

,Medicine,Chemistry,Physics,Biology,Psychology,Economics,Materials science,Philosophy,Engineering,Political science,History,Computer science,Sociology,Mathematics,Geology,Business,Art,Geography,Environmental science
Appenzeller-Herzog_2019,0.791792,0.006277,0.000000,0.008704,0.000000,0.000000,0.000000,0.004839,0.000000,0.000000,0.013664,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Bos_2018,0.731475,0.000000,0.000000,0.000000,0.134993,0.000000,0.000000,0.004561,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Brouwer_2019,0.211561,0.000000,0.000000,0.000000,0.602484,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chou_2003,0.847146,0.000000,0.000000,0.000000,0.007400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chou_2004,0.695533,0.000000,0.000000,0.000000,0.056843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Donners_2021,0.817031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Hall_2012,0.000729,0.000000,0.000000,0.000634,0.000000,0.000000,0.000000,0.000000,0.061725,0.000000,0.000000,0.694767,0.000000,0.022457,0.014671,0.000869,0.000000,0.000000,0.000000
Jeyaraman_2020,0.906323,0.004976,0.000000,0.001607,0.000000,0.000000,0.004009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Leenaars_2019,0.243685,0.245800,0.000000,0.154579,0.256428,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003783,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Leenaars_2020,0.865833,0.023698,0.000000,0.008905,0.000295,0.000000,0.001317,0.000289,0.000000,0.000000,0.000000,0.000511,0.000000,0.000449,0.000000,0.000097,0.000000,0.000000,0.000000


In [6]:
st = concepts_per_dataset.stack()
st = st[st.groupby(level=0).transform(max) == st].reset_index(level=1)
st.columns = ["concept_search", "score_search"]

st_incl = concepts_per_dataset_incl.stack()
st_incl = st_incl[st_incl.groupby(level=0).transform(max) == st_incl].reset_index(level=1)
st_incl.columns = ["concept_inclusions", "score_inclusions"]

concepts_pub = []
for d in sd.iter_datasets():
    r = {"name": d.name, "concept_publication": filter(lambda x: (x["level"] == 0), d.metadata["publication"]["concepts"]).__next__()["display_name"]}
    concepts_pub.append(r)
    
df_pub = pd.DataFrame(concepts_pub).set_index("name")

In [7]:
st_0n = concepts_per_dataset.stack()
st_0n[st_0n >=0.2] 


Appenzeller-Herzog_2019  Medicine             0.588982
Bos_2018                 Medicine             0.565684
                         Psychology           0.202546
Brouwer_2019             Medicine             0.332740
                         Psychology           0.451345
Chou_2003                Medicine             0.865703
Chou_2004                Medicine             0.760822
Donners_2021             Medicine             0.663195
Hall_2012                Computer science     0.599865
Jeyaraman_2020           Medicine             0.723767
Leenaars_2019            Medicine             0.233326
                         Chemistry            0.420204
Leenaars_2020            Medicine             0.839694
Meijboom_2021            Medicine             0.751780
Menon_2022               Medicine             0.675263
Moran_2021               Biology              0.360793
Muthu_2021               Medicine             0.919395
Nelson_2002              Medicine             0.794984
Oud_2018  

In [8]:
st_0n_incl = concepts_per_dataset_incl.stack()
st_0n_incl[st_0n_incl >=0.2] 


Appenzeller-Herzog_2019  Medicine            0.791792
Bos_2018                 Medicine            0.731475
Brouwer_2019             Medicine            0.211561
                         Psychology          0.602484
Chou_2003                Medicine            0.847146
Chou_2004                Medicine            0.695533
Donners_2021             Medicine            0.817031
Hall_2012                Computer science    0.694767
Jeyaraman_2020           Medicine            0.906323
Leenaars_2019            Medicine            0.243685
                         Chemistry           0.245800
                         Psychology          0.256428
Leenaars_2020            Medicine            0.865833
Meijboom_2021            Medicine            0.893434
Menon_2022               Medicine            0.605102
Moran_2021               Medicine            0.214867
                         Biology             0.338177
Muthu_2021               Medicine            0.950681
Nelson_2002              Med

In [9]:
df_pub.join(st["concept_search"]).join(st_incl["concept_inclusions"])

,concept_publication,concept_search,concept_inclusions
name,,,
Appenzeller-Herzog_2019,Medicine,Medicine,Medicine
Bos_2018,Medicine,Medicine,Medicine
Brouwer_2019,Psychology,Psychology,Psychology
Chou_2003,Medicine,Medicine,Medicine
Chou_2004,Medicine,Medicine,Medicine
Donners_2021,Medicine,Medicine,Medicine
Hall_2012,Computer science,Computer science,Computer science
Jeyaraman_2020,Medicine,Medicine,Medicine
Leenaars_2019,Medicine,Chemistry,Psychology
